## Start

In [1]:
import clickhouse_connect
import datetime
import os
import pytz
import pandas as pd
from dateutil.relativedelta import relativedelta
from dotenv import load_dotenv

import sys
sys.path.append('/home/boris/Documents/Work/analytics/Clickhouse')
from clickhouse_client import ClickHouse_client
ch = ClickHouse_client()
pd.set_option('display.max_rows', 1000)

___
### Tags: #Mobile_Report

### Links:

[[citizens_st_mobile]]

[[flats_dir_partner]]

[[entries_installation_points_dir_partner]]

[[installation_point_st_partner]]

[[companies_st_partner]]
___

In [10]:
query_text = """--sql
    SELECT
        citizens_st_mobile.report_date AS report_date,
        citizens_st_mobile.citizen_id AS citizen_id,
        citizens_st_mobile.trial_available AS trial_available,
        citizens_st_mobile.state AS state,
        citizens_st_mobile.flat_uuid AS flat_uuid,
        flats_dir_partner.address_uuid AS address_uuid,
        entries_installation_points_dir_partner.city AS city,
        installation_point_st_partner.partner_uuid AS partner_uuid,
        installation_point_st_partner.monetization AS monetization,
        installation_point_st_partner.monetization_is_allowed AS monetization_is_allowed,
        flats_dir_partner.created_at AS flat_created_at,
        citizens_st_mobile.activated_at AS activated_at
    FROM db1.citizens_st_mobile_ch AS citizens_st_mobile
    LEFT JOIN db1.flats_dir_partner_ch AS flats_dir_partner 
        ON flats_dir_partner.flat_uuid = citizens_st_mobile.flat_uuid
    LEFT JOIN db1.entries_installation_points_dir_partner_ch AS entries_installation_points_dir_partner 
        ON flats_dir_partner.address_uuid = entries_installation_points_dir_partner.address_uuid
    LEFT JOIN db1.installation_point_st_partner_ch AS installation_point_st_partner
        ON installation_point_st_partner.installation_point_id = entries_installation_points_dir_partner.installation_point_id
        AND installation_point_st_partner.report_date = citizens_st_mobile.report_date
    LEFT JOIN db1.companies_st_partner_ch AS companies_st_partner
        ON installation_point_st_partner.partner_uuid = companies_st_partner.partner_uuid
        AND companies_st_partner.report_date = citizens_st_mobile.report_date
    WHERE citizens_st_mobile.state = 'activated'
    limit 10
    """

ch.get_schema(query_text)

(
    `report_date` Date,
    `citizen_id` Int32,
    `trial_available` Int32,
    `state` String,
    `flat_uuid` String,
    `address_uuid` String,
    `city` String,
    `partner_uuid` String,
    `monetization` Int16,
    `monetization_is_allowed` Int16,
    `flat_created_at` DateTime,
    `activated_at` DateTime
)


In [13]:
query_text = """--sql
CREATE TABLE db1.rep_mobile_citizens_id_city_partner
(
    `report_date` Date,
    `citizen_id` Int32,
    `trial_available` Int32,
    `state` String,
    `flat_uuid` String,
    `address_uuid` String,
    `city` String,
    `partner_uuid` String,
    `monetization` Int16,
    `monetization_is_allowed` Int16,
    `flat_created_at` DateTime,
    `activated_at` DateTime
)
ENGINE = MergeTree()
ORDER BY citizen_id
"""

ch.query_run(query_text)

shape: (0, 0)
┌┐
╞╡
└┘

In [6]:
query_text = """--sql
    CREATE MATERIALIZED VIEW db1.rep_mobile_citizens_id_city_partner_mv
    REFRESH EVERY 1 DAY OFFSET 3 HOUR 55 MINUTE TO db1.rep_mobile_citizens_id_city_partner AS
   SELECT
        citizens_st_mobile.report_date AS report_date,
        citizens_st_mobile.citizen_id AS citizen_id,
        citizens_st_mobile.trial_available AS trial_available,
        citizens_st_mobile.state AS state,
        citizens_st_mobile.flat_uuid AS flat_uuid,
        flats_dir_partner.address_uuid AS address_uuid,
        entries_installation_points_dir_partner.city AS city,
        installation_point_st_partner.partner_uuid AS partner_uuid,
        installation_point_st_partner.monetization AS monetization,
        installation_point_st_partner.monetization_is_allowed AS monetization_is_allowed,
        flats_dir_partner.`created_at` AS flat_created_at,
        citizens_st_mobile_ch.activated_at AS activated_at
    FROM db1.citizens_st_mobile_ch AS citizens_st_mobile
    LEFT JOIN db1.flats_dir_partner_ch AS flats_dir_partner 
        ON flats_dir_partner.flat_uuid = citizens_st_mobile.flat_uuid
    LEFT JOIN db1.entries_installation_points_dir_partner_ch AS entries_installation_points_dir_partner 
        ON flats_dir_partner.address_uuid = entries_installation_points_dir_partner.address_uuid
    LEFT JOIN db1.installation_point_st_partner_ch AS installation_point_st_partner
        ON installation_point_st_partner.installation_point_id = entries_installation_points_dir_partner.installation_point_id
        AND installation_point_st_partner.report_date = citizens_st_mobile.report_date
    LEFT JOIN db1.companies_st_partner_ch AS companies_st_partner
        ON installation_point_st_partner.partner_uuid = companies_st_partner.partner_uuid
        AND companies_st_partner.report_date = citizens_st_mobile.report_date
    WHERE citizens_st_mobile.state = 'activated'
    """

ch.query_run(query_text)

""


___
## Tools
___
### query


In [4]:
query_text = """--sql
    SELECT
        *
    FROM db1.rep_mobile_citizens_id_city_partner
    ORDER BY report_date DESC
    limit 100
    """

ch.query_run(query_text)


,report_date,citizen_id,trial_available,state,flat_uuid,address_uuid,city,partner_uuid,monetization,monetization_is_allowed,flat_created_at,activated_at
0,2025-09-10,8444,0,activated,d1be1536-7b7d-4297-9a41-ac512ea81ed8,e245efbb-e3d4-4a5e-8b17-a15db50f97fd,село Осиново,dbeda098-942c-4071-899e-a76faf1c2691,1,1,2019-12-20 10:02:02,2019-12-20 13:43:16
1,2025-09-10,8443,0,activated,39131c5f-c9cf-409a-8d8b-67bfcb18a9de,0c1b865e-56ab-4c31-a26b-b94d923294f4,Ульяновск,fdae9241-37f2-4446-866c-cd9119425fe4,1,1,2019-12-20 14:51:10,2019-12-20 17:50:51
2,2025-09-10,8442,0,activated,8b3cc1f2-281c-45bc-829c-0f0446dcd012,0c1b865e-56ab-4c31-a26b-b94d923294f4,Ульяновск,fdae9241-37f2-4446-866c-cd9119425fe4,1,1,2019-12-20 12:50:31,2019-12-20 15:50:17
3,2025-09-10,8441,0,activated,430a1ab6-89f4-4971-aa7c-a7f19f214c37,2e04e3c7-6060-4ae8-81cc-43ad58a7f6e4,Тюмень,adeffb47-1707-4976-a4a6-e6eacf19b050,1,1,2019-12-20 10:43:53,2019-12-20 13:43:02
4,2025-09-10,8440,0,activated,d1be1536-7b7d-4297-9a41-ac512ea81ed8,e245efbb-e3d4-4a5e-8b17-a15db50f97fd,село Осиново,dbeda098-942c-4071-899e-a76faf1c2691,1,1,2019-12-20 10:02:02,2019-12-20 13:01:35
5,2025-09-10,8439,0,activated,00b2df8f-618f-494b-b842-8b1c53dc0deb,a15d8f64-8dd5-4886-a20c-c3eb5cd6f35b,Якутск,6ee3f2e2-708c-4f9b-b997-3c806c5468f6,1,1,2019-12-18 06:00:50,2019-12-20 12:47:10
6,2025-09-10,8438,0,activated,0f9e858f-e049-439a-9136-e00eb99b9f6a,2fcf2651-e489-498f-a6e3-84a2f509e60a,Обнинск,3ca65195-dfda-41f5-9cbf-1bf841748ffc,1,1,2019-12-20 09:07:22,2019-12-20 12:07:00
7,2025-09-10,8436,0,activated,bcce27fe-ba82-4d9e-b5d8-d3cd50a744f4,efece0c5-f6b8-4338-81b0-fd3ceee135ec,Семей,af8b5bf4-0108-4a0c-acc2-096f94959fb3,0,0,2019-12-30 11:18:26,2024-12-28 15:11:39
8,2025-09-10,8435,0,activated,1368342c-75d8-42d6-acbb-dbb5598cee8c,7485c497-7507-40cb-8a8c-e7f9303b8c13,Якутск,6ee3f2e2-708c-4f9b-b997-3c806c5468f6,1,1,2020-05-25 03:34:27,2020-05-21 10:16:55
9,2025-09-10,8434,0,activated,3d13d43f-9513-4f3f-8a95-efad55b6c740,0c1b865e-56ab-4c31-a26b-b94d923294f4,Ульяновск,fdae9241-37f2-4446-866c-cd9119425fe4,1,1,2020-02-14 06:46:31,2020-08-07 12:17:00


### delete a part


In [ ]:
query_text = """--sql
    ALTER TABLE db1.rep_mobile_citizens_id_city_partner DELETE WHERE report_date = '2025-07-17'
    """

ch.query_run(query_text)


### drop mv

In [5]:
query_text = """--sql
    DROP TABLE db1.rep_mobile_citizens_id_city_partner_mv
    """

ch.query_run(query_text)

""


### drop ch

In [12]:
query_text = """--sql
    DROP TABLE db1.rep_mobile_citizens_id_city_partner
    """

ch.query_run(query_text)

shape: (0, 0)
┌┐
╞╡
└┘

### refresh mv

In [3]:
query_text = """
SYSTEM REFRESH VIEW db1.rep_mobile_citizens_id_city_partner_mv
"""

ch.query_run(query_text)

,read_rows,read_bytes,written_rows,written_bytes,total_rows_to_read,result_rows,result_bytes,elapsed_ns,query_id
0,0,0,0,0,0,0,0,836994,a56b8b93-adaa-427c-9db2-5da5a3ca9ba9
